In [1]:
import pickle
import torch

In [ ]:
# probs32 and probs16 are single tensors of shape [num_samples * batch, vocab_size]
# Load them first if needed
with open("prop_output32.pkl", "rb") as f:
    probs32 = pickle.load(f)  # shape [total_samples, vocab_size]

with open("prop_output16.pkl", "rb") as f:
    probs16 = pickle.load(f)  # shape [total_samples, vocab_size]

min_samples = min(probs32.shape[0], probs16.shape[0])
probs32 = probs32[:min_samples, :]
probs16 = probs16[:min_samples, :]

# Compute predicted token IDs
pred32 = torch.argmax(probs32, dim=-1)  # shape [total_samples]
pred16 = torch.argmax(probs16, dim=-1)  # shape [total_samples]

# Compare
matches = (pred32 == pred16).sum().item()
total = pred32.shape[0]
accuracy = matches / total

print(f"✅ Next-token agreement between float32 and float16: {accuracy*100:.2f}%")

Shape of probs32: torch.Size([1602, 50280])
Shape of probs16: torch.Size([1602, 50280])
✅ Next-token agreement between float32 and float16: 98.81%


In [3]:

min_samples = min(probs32.shape[0], probs16.shape[0])
probs32 = probs32[:min_samples, :]
probs16 = probs16[:min_samples, :]

# 🔹 Choose top-k
k = 5  # change this to any k you want

# Compute top-k token indices
values32, indices32 = torch.topk(probs32, k=k, dim=-1)
values16, indices16 = torch.topk(probs16, k=k, dim=-1)

# 🔹 Compute top-k set overlap (regardless of order)
overlap_counts = [
    len(set(indices32[i].tolist()) & set(indices16[i].tolist()))
    for i in range(indices32.shape[0])
]
overlap_ratio = sum(overlap_counts) / (indices32.shape[0] * k)

# 🔹 Compute exact top-k order match (same tokens in same positions)
exact_match = (indices32 == indices16).all(dim=-1).float().mean().item()

# 🔹 Print results
print(f"✅ Top-{k} overlap ratio (same tokens, any order): {overlap_ratio*100:.2f}%")
print(f"✅ Top-{k} exact ordering match: {exact_match*100:.2f}%")

# (Optional) For reference, still compute top-1 agreement
pred32 = torch.argmax(probs32, dim=-1)
pred16 = torch.argmax(probs16, dim=-1)
accuracy = (pred32 == pred16).float().mean().item()
print(f"✅ Top-1 (argmax) agreement: {accuracy*100:.2f}%")

✅ Top-5 overlap ratio (same tokens, any order): 99.03%
✅ Top-5 exact ordering match: 89.01%
✅ Top-1 (argmax) agreement: 98.81%
